# Environment Setup

In [1]:
!uv pip install bitsandbytes xformers triton unsloth vllm==0.10.2
!uv pip install transformers==4.55.4
!uv pip install --no-deps trl==0.22.2

Using Python 3.12.11 environment at: /usr
Resolved 166 packages in 1.35s
Prepared 47 packages in 8.68s
Uninstalled 6 packages in 123ms
Installed 47 packages in 80ms
 + astor==0.8.1
 + bitsandbytes==0.48.1
 + blake3==1.0.7
 + cbor2==5.7.0
 + compressed-tensors==0.11.0
 + cut-cross-entropy==25.1.1
 - datasets==4.0.0
 + datasets==4.1.1
 + depyf==0.19.0
 + diskcache==5.6.3
 + dnspython==2.8.0
 + email-validator==2.3.0
 + fastapi-cli==0.0.13
 + fastapi-cloud-cli==0.3.0
 + gguf==0.17.1
 + httptools==0.6.4
 + interegular==0.3.3
 - lark==1.3.0
 + lark==1.2.2
 + llguidance==0.7.30
 - llvmlite==0.43.0
 + llvmlite==0.44.0
 + lm-format-enforcer==0.11.3
 + mistral-common==1.8.5
 + msgspec==0.19.0
 + ninja==1.13.0
 - numba==0.60.0
 + numba==0.61.2
 + openai-harmony==0.0.4
 + outlines-core==0.2.11
 + partial-json-parser==0.2.1.1.post6
 + prometheus-fastapi-instrumentator==7.1.0
 - pyarrow==18.1.0
 + pyarrow==21.0.0
 + pybase64==1.4.2
 + pycountry==24.6.1
 + pydantic-extra-types==2.10.5
 + ray==2.49.2

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/MyDrive/multi-reward-math-reasoning # Add this folder as shortcut to your Drive to save your results here
!ls

In [2]:
from unsloth import FastLanguageModel
from trl import SFTConfig, GRPOConfig, SFTTrainer, GRPOTrainer
from vllm import SamplingParams

import gc
import re
import time
import torch
import numpy as np
import pandas as pd

from pathlib import Path
from tqdm.notebook import tqdm
from datasets import load_dataset, Dataset
from safetensors import safe_open

from peft import LoraConfig, get_peft_model, TaskType
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    BitsAndBytesConfig, TextStreamer
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 10-07 17:19:28 [__init__.py:216] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!


# Model Setup

In [3]:
model_id = 'unsloth/Qwen3-1.7B-Base'          # Select model optimized for instruction-following and reasoning
model_name = model_id.split('/')[-1].lower()  # Extract model name from ID
max_seq_length = 2048                         # Can increase for longer reasoning traces
lora_rank = 32                                # Larger rank = smarter, but slower

In [4]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_id,
    max_seq_length=max_seq_length,
    load_in_4bit=False,         # False for LoRA 16bit
    fast_inference=True,        # Enable vLLM fast inference
    max_lora_rank=lora_rank,
    gpu_memory_utilization=0.9, # Reduce if out of memory
)
model = FastLanguageModel.get_peft_model(
    model,
    r=lora_rank,                          # Rank: adaptation capacity (16 good for reasoning tasks)
    lora_alpha=lora_rank * 2,             # Scaling factor (typically 2x rank)
    lora_dropout=0.1,                     # Regularization to prevent overfitting
    target_modules=[                      # Remove QKVO if out of memory
        'q_proj', 'k_proj', 'v_proj', 'o_proj',
        'gate_proj', 'up_proj', 'down_proj',
    ],
    use_gradient_checkpointing='unsloth', # Reduces memory usage
    random_state=3407,
)

INFO 10-07 17:19:41 [vllm_utils.py:689] Unsloth: Patching vLLM v1 graph capture
INFO 10-07 17:19:41 [vllm_utils.py:717] Unsloth: Patching vLLM v0 graph capture
==((====))==  Unsloth 2025.10.1: Fast Qwen3 patching. Transformers: 4.55.4. vLLM: 0.10.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/Qwen3-1.7B-Base with actual GPU utilization = 88.97%
Unsloth: Your GPU has CUDA compute capability 8.0 with VRAM = 39.56 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 2048. Num Sequences = 320.
Unsloth: vLLM's KV Cache can use up to 31.92 GB. Also swap space = 6 GB.
WARNING 10-07 17:19:44 [compilation.py

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/617 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.1.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.10.1 patched 28 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


# Chat Template

In [5]:
# Define structured output format for mathematical reasoning
REASONING_START = '<THINK>'   # Begin reasoning section
REASONING_END = '</THINK>'    # End reasoning section
SOLUTION_START = '<SOLUTION>' # Begin final answer
SOLUTION_END = '</SOLUTION>'  # End final answer

# System prompt that teaches the model our desired reasoning structure
SYSTEM_PROMPT = f'''You are a mathematical reasoning assistant. When given a math problem:
1. Show your step-by-step work between {REASONING_START} and {REASONING_END}.
2. Provide your final numerical answer between {SOLUTION_START} and {SOLUTION_END}.
3. Be precise and show all calculation steps clearly.'''
print(SYSTEM_PROMPT)

You are a mathematical reasoning assistant. When given a math problem:
1. Show your step-by-step work between <THINK> and </THINK>.
2. Provide your final numerical answer between <SOLUTION> and </SOLUTION>.
3. Be precise and show all calculation steps clearly.


In [6]:
chat_template = ( # Build and assign chat_template to the tokenizer
    # If the very first message is a SYSTEM role, print it + <eos>:
    "{% if messages[0]['role'] == 'system' %}"
      "{{ messages[0]['content'] + eos_token }}"
      "{% set loop_messages = messages[1:] %}"
    "{% else %}"
      # Otherwise, inject our system_prompt + <eos>:
      "{{ '{system_prompt}' + eos_token }}"
      "{% set loop_messages = messages %}"
    "{% endif %}"

    # Now loop over the remaining messages (either user or assistant):
    "{% for message in loop_messages %}"
      "{% if message['role'] == 'user' %}"
        "{{ message['content'] }}"
      "{% elif message['role'] == 'assistant' %}"
        "{{ message['content'] + eos_token }}"
      "{% endif %}"
    "{% endfor %}"

    # If we asked for "add_generation_prompt", append <REASONING> to the end:
    "{% if add_generation_prompt %}{{ '{reasoning_start}' }}"
    "{% endif %}"
)
# Replace with out specific template:
tokenizer.chat_template = chat_template\
    .replace("'{system_prompt}'",   f"'{SYSTEM_PROMPT}'")\
    .replace("'{reasoning_start}'", f"'{REASONING_START}'")

In [7]:
example_messages = [ # Quick sanity check of the template
    {'role': 'user', 'content': 'Which country has the highest population density?'},
    {'role': 'assistant', 'content': (
        f'{REASONING_START}'
        'I know that country X is small in area but has a huge population, '
        'so its people per square kilometer is extremely high.'
        f'{REASONING_END}{SOLUTION_START}Monaco{SOLUTION_END}'
    )},
    {'role': 'user', 'content': 'Which planet is farthest from the Sun?'},
]
print(tokenizer.apply_chat_template(example_messages, tokenize=False, add_generation_prompt=True))

You are a mathematical reasoning assistant. When given a math problem:
1. Show your step-by-step work between <THINK> and </THINK>.
2. Provide your final numerical answer between <SOLUTION> and </SOLUTION>.
3. Be precise and show all calculation steps clearly.<|endoftext|>Which country has the highest population density?<THINK>I know that country X is small in area but has a huge population, so its people per square kilometer is extremely high.</THINK><SOLUTION>Monaco</SOLUTION><|endoftext|>Which planet is farthest from the Sun?<THINK>


# Pre Fine-tuning (SFT)

## Data preparation

In [8]:
# Use a subset of NVIDIA's Open Math Reasoning dataset, which was filtered to only include high quality DeepSeek R1 traces
sft_dataset = load_dataset('unsloth/OpenMathReasoning-mini', split='cot').to_pandas()
sft_dataset = sft_dataset[['expected_answer', 'problem', 'generated_solution']]

# Try converting to number - if not, replace with NaN
is_number = pd.to_numeric(pd.Series(sft_dataset['expected_answer']), errors='coerce').notnull()
sft_dataset = sft_dataset.iloc[np.where(is_number)[0]] # Select only numbers
sft_dataset

README.md:   0%|          | 0.00/603 [00:00<?, ?B/s]

data/cot-00000-of-00001.parquet:   0%|          | 0.00/106M [00:00<?, ?B/s]

Generating cot split:   0%|          | 0/19252 [00:00<?, ? examples/s]

,expected_answer,problem,generated_solution
0,14,Given $\sqrt{x^2+165}-\sqrt{x^2-52}=7$ and $x$...,"<think>\nOkay, let's see. I need to solve the ..."
6,-2,Find the value of the parameter $a$ for which ...,"<think>\nOkay, so I need to find the value of ..."
9,18,What is the sum of all real numbers $x$ for wh...,"<think>\nOkay, so I need to solve the equation..."
13,2,Evaluate the sum \(\sum_{n=1}^\infty \frac{\ph...,"<think>\nOkay, so I need to evaluate the infin..."
17,30,What is the largest positive integer that divi...,"<think>\nAlright, so I need to find the larges..."
...,...,...,...
19243,244,"Let \( p \), \( q \), and \( r \) be the disti...","<think>\nOkay, so I need to find the value of ..."
19245,1,A bug is on the $0$ of a number line. At any p...,"<think>\nOkay, so I have this problem where a ..."
19247,4,A bus left point X for point Y. Two hours late...,"<think>\nOkay, let's tackle this problem step ..."
19248,18,Each interior angle of a regular n-gon measure...,"<think>\nOkay, let's see. I need to find the n..."


In [9]:
def format_dataset(x): # Format the dataset to follow our GRPO style formatting
    expected_answer = x['expected_answer']
    problem = x['problem']

    # Remove generated <think> and </think>
    thoughts = x['generated_solution'].replace('<think>', '').replace('</think>', '')
    thoughts = thoughts.strip()

    # Add our custom formatting
    final_prompt = REASONING_START + thoughts + REASONING_END + \
                   SOLUTION_START + expected_answer + SOLUTION_END
    return [
        {'role': 'system'   , 'content': SYSTEM_PROMPT},
        {'role': 'user'     , 'content': problem},
        {'role': 'assistant', 'content': final_prompt},
    ]

sft_dataset['messages'] = sft_dataset.apply(format_dataset, axis=1)
print(tokenizer.apply_chat_template(sft_dataset['messages'][0], tokenize=False))

You are a mathematical reasoning assistant. When given a math problem:
1. Show your step-by-step work between <THINK> and </THINK>.
2. Provide your final numerical answer between <SOLUTION> and </SOLUTION>.
3. Be precise and show all calculation steps clearly.<|endoftext|>Given $\sqrt{x^2+165}-\sqrt{x^2-52}=7$ and $x$ is positive, find all possible values of $x$.<THINK>Okay, let's see. I need to solve the equation √(x² + 165) - √(x² - 52) = 7, and find all positive values of x. Hmm, radicals can be tricky, but maybe if I can eliminate the square roots by squaring both sides. Let me try that.

First, let me write down the equation again to make sure I have it right:

√(x² + 165) - √(x² - 52) = 7.

Okay, so the idea is to isolate one of the radicals and then square both sides. Let me try moving the second radical to the other side:

√(x² + 165) = 7 + √(x² - 52).

Now, if I square both sides, maybe I can get rid of the square roots. Let's do that:

(√(x² + 165))² = (7 + √(x² - 52))².

Sim

In [10]:
# Truncate pre fine-tuning sft_dataset to max_seq_length / 2 since we don't want too long reasoning traces
sft_dataset['seq_length'] = sft_dataset['messages'].apply(lambda x: len(tokenizer.apply_chat_template(x)))
print('Token-length percentiles (50/90/99):', np.percentile(sft_dataset['seq_length'], [50, 90, 99]))

threshold = max_seq_length / 2
sft_dataset_filtered = sft_dataset.loc[sft_dataset['seq_length'] <= threshold].copy()
print(f'Remaining for training (<= {threshold} tokens): {len(sft_dataset_filtered)}/{len(sft_dataset)}')

sft_dataset_filtered['text'] = tokenizer.apply_chat_template(sft_dataset_filtered['messages'].values.tolist(), tokenize=False)
sft_dataset_filtered = Dataset.from_pandas(sft_dataset_filtered)
sft_dataset_filtered

Token-length percentiles (50/90/99): [ 3729.    9034.   15685.84]
Remaining for training (<= 1024.0 tokens): 51/7507


Dataset({
    features: ['expected_answer', 'problem', 'generated_solution', 'messages', 'seq_length', 'text', '__index_level_0__'],
    num_rows: 51
})

## Pre fine-tune to understand custom GRPO formatting

In [11]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=sft_dataset_filtered,
    args=SFTConfig(
        dataset_text_field='text',
        num_train_epochs=3,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=1,
        optim='adamw_8bit',
        weight_decay=0.01,
        learning_rate=2e-4,
        lr_scheduler_type='cosine',
        warmup_steps=5,
        logging_steps=5,
        seed=3407,
        report_to='none', # Use this for WandB
    )
)
trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/51 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 51 | Num Epochs = 3 | Total steps = 153
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 1 x 1) = 1
 "-____-"     Trainable parameters = 34,865,152 of 1,755,440,128 (1.99% trained)


Step,Training Loss
5,0.873000
10,0.751400
15,0.582600
20,0.497600
25,0.520600
30,0.460200
35,0.399700
40,0.474800
45,0.449800
50,0.453700


Unsloth: Will smartly offload gradients to save VRAM!


TrainOutput(global_step=153, training_loss=0.3443734077067157, metrics={'train_runtime': 78.7979, 'train_samples_per_second': 1.942, 'train_steps_per_second': 1.942, 'total_flos': 1196951737651200.0, 'train_loss': 0.3443734077067157, 'epoch': 3.0})

## Check if model has learnt to follow the format

In [12]:
text = tokenizer.apply_chat_template( # Render into a single string and append <REASONING> for generation
    sft_dataset_filtered[1]['messages'][:2],
    tokenize=False, add_generation_prompt=True, # Append the final <REASONING>
)
_ = model.generate(
    **tokenizer(text, return_tensors='pt').to('cuda'),
    temperature=0, max_new_tokens=1024,
    streamer=TextStreamer(tokenizer, skip_prompt=False), # Stream the model's generations (CoT + solution)
)

You are a mathematical reasoning assistant. When given a math problem:
1. Show your step-by-step work between <THINK> and </THINK>.
2. Provide your final numerical answer between <SOLUTION> and </SOLUTION>.
3. Be precise and show all calculation steps clearly.<|endoftext|>What is the average book width, in centimeters, of five books with the following widths: $6$, $\frac{1}{2}$, $1$, $2.5$, and $10$?<THINK>Okay, let's see. I need to find the average width of five books. The widths given are 6 cm, 1/2 cm, 1 cm, 2.5 cm, and 10 cm. Hmm, average is when you add up all the numbers and then divide by how many there are. So first, I should add these numbers together. Let me write them down: 6, 0.5 (which is 1/2), 1, 2.5, and 10. 

Wait, adding decimals can be tricky. Maybe I should convert all the numbers to decimals to make it easier. 1/2 is 0.5, so the numbers are 6, 0.5, 1, 2.5, and 10. Let me add them step by step. Starting with 6 and 0.5. 6 plus 0.5 is 6.5. Then add 1: 6.5 + 1 = 7.5. Nex

In [13]:
del sft_dataset, sft_dataset_filtered
gc.collect()
torch.cuda.empty_cache()

# Post Fine-tuning (RL)

## Data preparation

In [14]:
def process_dataset_sample(example): # Convert GSM8K example to conversation format for GRPO training
    return {
        'prompt': [ # Create conversation with system prompt for structured reasoning
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': example['question']},
        ],
        # Extract numerical answer from GSM8K format ('Explanation... #### 42') as Ground truth for reward functions
        'answer': example['answer'].split('####')[1].strip() if '####' in example['answer'] else None
    }

In [15]:
# train_dataset = load_dataset('openai/gsm8k', 'main', split=['train[:10%]'])
train_dataset = load_dataset('openai/gsm8k', 'main', split='train')
train_dataset = train_dataset.map(process_dataset_sample)

print(f'Training samples: {len(train_dataset):,}\n'
      f"- Sample question: {train_dataset[0]['prompt'][1]['content']}\n"
      f"- Sample answer (ground truth for rewards): {train_dataset[0]['answer']}\n"
      f"- Prompt (system + user):\n{train_dataset[0]['prompt']}")

README.md: 0.00B [00:00, ?B/s]

main/train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

main/test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

Training samples: 7,473
- Sample question: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
- Sample answer (ground truth for rewards): 72
- Prompt (system + user):
[{'content': 'You are a mathematical reasoning assistant. When given a math problem:\n1. Show your step-by-step work between <THINK> and </THINK>.\n2. Provide your final numerical answer between <SOLUTION> and </SOLUTION>.\n3. Be precise and show all calculation steps clearly.', 'role': 'system'}, {'content': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?', 'role': 'user'}]


In [16]:
# Get the top 90% prompt length so we don't accidentally truncate them, i.e. we'll remove the top 10% long prompts
tokenized_dataset = train_dataset.map(
    lambda x: {'tokens': tokenizer.apply_chat_template(x['prompt'], add_generation_prompt=True, tokenize=True)},
    batched=True,
).map(lambda x: {'length': len(x['tokens'])})
print(tokenizer.decode(tokenized_dataset[0]['tokens']))

thresholds = np.percentile(tokenized_dataset['length'], [50, 90, 99])
max_prompt_length = int(thresholds[1])
print('Token-length percentiles (50/90/99):', thresholds, '=> Choose max_prompt_length =', max_prompt_length)

# Filter only samples smaller than 90% max length
train_dataset = train_dataset.select(np.where(np.array(tokenized_dataset['length']) <= max_prompt_length)[0])
print(f'Remaining for training (<= {max_prompt_length} tokens): {len(train_dataset)}/{len(tokenized_dataset)}')
del tokenized_dataset

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

Map:   0%|          | 0/7473 [00:00<?, ? examples/s]

You are a mathematical reasoning assistant. When given a math problem:
1. Show your step-by-step work between <THINK> and </THINK>.
2. Provide your final numerical answer between <SOLUTION> and </SOLUTION>.
3. Be precise and show all calculation steps clearly.<|endoftext|>Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?<THINK>
Token-length percentiles (50/90/99): [119. 152. 191.] => Choose max_prompt_length = 152
Remaining for training (<= 152 tokens): 6749/7473


## Regex Patterns

In [17]:
# Match the reasoning sections and answers
match_format = re.compile(
    # rf'^[\s]{{0,}}'                                     # Optional whitespace at start
    # rf'{REASONING_START}.+?{REASONING_END}.*?'          # Reasoning section (non-greedy)
    rf'{REASONING_END}.*?'                              # We always prepend REASONING_START
    rf'{SOLUTION_START}(.+?){SOLUTION_END}'             # Solution section with capture group
    rf'[\s]{{0,}}(?:{re.escape(tokenizer.eos_token)})?' # Add optional EOS token matching
    rf'[\s]{{0,}}$',                                    # Optional whitespace at end
    flags=re.MULTILINE | re.DOTALL,                     # Multi-line matching with . matching newlines
)
match_format.findall( # Verify it works
    f'{REASONING_START}Let me think!{REASONING_END}'\
    f'{SOLUTION_START}\n2\n{SOLUTION_END}\n\n',
)

['\n2\n']

In [18]:
# Sometimes it might not be 1 number as the answer, but like a sentence.
# For example: 'The solution is $20' -> we extract 20
# We also remove possible commas for example as in 123,456
match_number = re.compile(
    rf'{SOLUTION_START}.*?[\s]{{0,}}([-]?[\d\.\,]{{1,}})', # Extract numbers from solution section
    flags=re.MULTILINE | re.DOTALL | re.IGNORECASE,        # Flexible pattern matching
)
print(match_number.findall('<SOLUTION>  0.34  </SOLUTION>'))
print(match_number.findall('<SOLUTION>  123,456  </SOLUTION>'))
print(match_number.findall('<SOLUTION>  -0.234  </SOLUTION>'))
print(match_number.findall('<SOLUTION>17</SOLUTION>'))

['0.34']
['123,456']
['-0.234']
['17']


## Multi-reward design

In [19]:
def match_format_strictly(completions, **kwargs) -> list[float]:
    ''' Reward Function 1: Exact Format Compliance
    High reward (3.0) for perfect format adherence
    Ensures model learns the complete structured output pattern
    '''
    return [
        3.0 if match_format.search(completion[0]['content']) else 0.0
        for completion in completions
    ]

In [20]:
# If it fails, reward the model if it at least follows the format partially, by counting each symbol
def match_format_softly(completions, **kwargs) -> list[float]:
    ''' Reward Function 2: Partial Format Credit
    Graduated scoring for format elements
    Encourages learning individual components even if not perfect
    '''
    rewards = []
    for completion in completions:
        reward = 0
        response = completion[0]['content']

        # Count how many keywords are seen - we penalize if too many!
        # Award +0.5 for correct token count, -0.5 for wrong count
        # No need to reward REASONING_START since we always prepend it!
        # reward += 0.5 if response.count(REASONING_START) == 1 else -0.5
        reward += 0.5 if response.count(REASONING_END) == 1 else -0.5
        reward += 0.5 if response.count(SOLUTION_START) == 1 else -0.5
        reward += 0.5 if response.count(SOLUTION_END) == 1 else -0.5
        rewards.append(reward)
    return rewards

In [21]:
# Extract the generated answer, and reward or penalize it
def check_answer_correctness(completions, answer, **kwargs) -> list[float]:
    ''' Reward Function 3: Graduated scoring for mathematical accuracy
    - 5.0: Exact string match gets full points
    - 2.0: Within 10% (close answer)
    - 1.5: Within 20% (reasonable attempt)
    - -2.5: Wrong answer (penalty for incorrect math)
    '''
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [ # Extract answers using format pattern
        guess.group(1) if (guess := match_format.search(r)) else None
        for r in responses
    ]
    rewards = []
    for guess, true_answer in zip(extracted_responses, answer):
        if guess is None: # No extractable answer
            rewards.append(-2.0)
            continue

        if guess == true_answer: rewards.append(5.0)                   # Correct answer gets 5 points!
        elif guess.strip() == true_answer.strip(): rewards.append(3.5) # Match if spaces are seen, but less reward
        else: # Try numerical comparison for partial credit
            try: # We also reward it based on how close the answer is to the true one via ratios
                ratio = float(guess) / float(true_answer)     # If the answer is within some range, reward it!
                if 0.9 <= ratio <= 1.1: rewards.append(2.0)   # Within 10%
                elif 0.8 <= ratio <= 1.2: rewards.append(1.5) # Within 20%
                else: rewards.append(-2.5)                    # Penalize wrong answers
            except (ValueError, ZeroDivisionError):
                rewards.append(-4.5)                          # Invalid numerical format
    return rewards

In [22]:
def check_numbers_extraction(prompts, completions, answer, **kwargs) -> list[float]:
    ''' Reward Function 4: Number Extraction Ability
    Tests the model's ability to extract numerical values from solution sections
    Complementary to exact format matching - focuses on parsing capability
    '''
    question = prompts[0][-1]['content'] # Exclude system prompt
    responses = [completion[0]['content'] for completion in completions]

    extracted_responses = [ # Extract numbers from solution sections using number pattern
        guess.group(1) if (guess := match_number.search(r)) else None
        for r in responses
    ]
    rewards = []

    # Print only every few steps
    check_numbers_extraction.counter = getattr(check_numbers_extraction, 'counter', 0) + 1
    if check_numbers_extraction.counter % 100 == 0:
        print(
            '==' * 100,
            f'\nQuestion: {question}'
            f'\nPrediction: {extracted_responses[0]}, GT Answer: {answer[0]}'
            f'\nResponse:\n{responses[0]}'
        )
    for guess, true_answer in zip(extracted_responses, answer):
        if guess is None: # No extractable number
            rewards.append(-2.5)
            continue

        try: # Simple numerical equality check
            true_val = float(true_answer.strip())             # Convert to numbers
            guess_val = float(guess.strip().replace(',', '')) # Remove commas like in 123,456
            rewards.append(3.5 if guess_val == true_val else -1.5)
        except (ValueError, TypeError):
            rewards.append(0) # Invalid number format
    return rewards

## GRPO training setup

In [23]:
max_prompt_length = 152 + 1 # + 1 just in case!
max_completion_length = max_seq_length - max_prompt_length
vllm_sampling_params = SamplingParams(
    min_p = 0.1,
    top_p = 1.0,
    top_k = -1,
    stop = [tokenizer.eos_token],
    include_stop_str_in_output = True,
)

In [24]:
training_args = GRPOConfig(          # Configure GRPO training parameters for mathematical reasoning
    output_dir=f'/tmp/{model_name}', # Directory for checkpoints and logs
    vllm_sampling_params=vllm_sampling_params,
    # Training speed control
    num_train_epochs=1,              # Total number of training epochs
    per_device_train_batch_size=2,   # Small batch for GPU memory constraints
    gradient_accumulation_steps=8,   # Effective batch size = 2 * 8 = 16
    # Computing the loss: https://huggingface.co/docs/trl/main/grpo_trainer#computing-the-loss
    scale_rewards='batch',           # Calculate mean at local/group level and std at global/batch level enables more robust reward shaping
    loss_type='dr_grpo',             # Fully remove response length bias, dividing by a constant instead of the sequence length
    # Precision & Optimization
    optim='adamw_8bit',              # adamw_torch_fused, adamw_8bit, paged_adamw_8bit
    weight_decay=0.1,                # Regularization
    max_grad_norm=0.1,               # Aggressive gradient clipping for stable training
    gradient_checkpointing=True,
    bf16=torch.cuda.is_available(),  # Enable mixed-precision training if a CUDA GPU is available (faster, less memory)
    # Learning rate scheduling
    learning_rate=1e-5,              # Conservative LR to prevent destabilizing reasoning
    warmup_ratio=0.1,
    lr_scheduler_type='cosine_with_min_lr',
    lr_scheduler_kwargs=dict(min_lr=1e-6),
    # Generation control
    temperature=1.0,
    num_generations=2,                           # Default: 8 generations per step
    max_prompt_length=max_prompt_length,         # Default: 512. Sufficient for complex word problems
    max_completion_length=max_completion_length, # Default: 256. Room for detailed step-by-step reasoning
    # Reporting and saving
    report_to='wandb',
    logging_steps=10,
    logging_strategy='steps',
    save_total_limit=1,
    # max_steps=100,
    # For optional evaluation
    # per_device_eval_batch_size=4,
    # bf16_full_eval=torch.cuda.is_available(),
    # eval_strategy='steps',                       # Evaluate after each epoch
    # load_best_model_at_end=True,                 # Load the best model based on validation loss
)

## Train the model

In [25]:
%%time
trainer = GRPOTrainer(            # Initialize GRPO trainer with multi-reward system
    model=model,                  # LoRA-adapted quantized model
    processing_class=tokenizer,
    train_dataset=train_dataset,  # Processed GSM8K dataset
    args=training_args,           # Training configuration
    reward_funcs=[                # 4 complementary reward functions
        match_format_strictly,    # Perfect structure compliance
        match_format_softly,      # Partial format credit
        check_answer_correctness, # Mathematical accuracy
        check_numbers_extraction, # Number parsing ability
    ]
)
trainer.train()
trainer.save_model(f'./{model_name}_grpo')

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 6,749 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 34,865,152 of 1,755,440,128 (1.99% trained)


Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / match_format_strictly / mean,rewards / match_format_strictly / std,rewards / match_format_softly / mean,rewards / match_format_softly / std,rewards / check_answer_correctness / mean,rewards / check_answer_correctness / std,rewards / check_numbers_extraction / mean,rewards / check_numbers_extraction / std
10,0.180000,11.246875,4.440526,833.106250,455.900000,1760.800000,0.031250,797.704602,455.900000,1498.200000,0.723357,2.887500,0.402470,1.406250,0.375000,4.078125,2.269380,2.875000,1.588505
20,0.052000,10.984375,4.095429,813.762500,459.000000,1580.900000,0.012500,800.383759,459.000000,1486.600000,0.309582,2.962500,0.150000,1.462500,0.150000,3.878125,2.319421,2.681250,1.676520
30,0.055900,11.468750,3.804934,837.675000,491.400000,1531.200000,0.037500,797.736090,491.400000,1377.800000,0.359620,2.868750,0.429939,1.387500,0.354939,4.281250,1.804838,2.931250,1.356275
40,0.149700,11.131250,4.382024,840.750000,460.800000,1662.500000,0.043750,794.928058,460.800000,1541.300000,0.286208,2.868750,0.429939,1.368750,0.429939,4.093750,2.107804,2.800000,1.632391
50,0.275000,10.731250,4.850727,832.606250,433.800000,1716.800000,0.050000,776.427472,433.800000,1346.900000,0.286255,2.850000,0.504939,1.350000,0.504939,3.906250,2.282245,2.625000,1.741736
60,0.231100,10.337500,5.430252,814.925000,413.700000,1698.700000,0.043750,765.332520,413.700000,1430.800000,0.292500,2.850000,0.504939,1.362500,0.454939,3.600000,2.769838,2.525000,2.029531
70,0.042300,11.234375,4.259282,791.006250,402.600000,1526.700000,0.018750,770.351318,402.600000,1490.800000,0.306503,2.943750,0.177470,1.443750,0.177470,3.984375,2.400032,2.862500,1.561585
80,0.122700,11.012500,4.297701,768.762500,421.900000,1436.800000,0.018750,748.189594,421.900000,1270.900000,0.281320,2.962500,0.150000,1.443750,0.225000,3.931250,2.296932,2.675000,1.763537
90,0.184800,10.815625,4.776361,757.631250,429.000000,1578.100000,0.037500,713.820850,429.000000,1276.000000,0.282309,2.868750,0.525000,1.381250,0.475000,3.915625,2.293364,2.650000,1.825312
100,0.108100,12.153125,2.714175,790.731250,427.000000,1424.700000,0.018750,770.580841,427.000000,1305.700000,0.279869,2.925000,0.300000,1.425000,0.300000,4.609375,1.236171,3.193750,0.954771


Question: James wants to build a 16-foot by 20-foot quilt.  He uses patches that are each 4 square feet.  The first 10 patches cost $10 each and then each patch after that cost half as much.  How much do the patches for the quilt cost?
Prediction: 450, GT Answer: 450
Response:
Okay, let's see. James is building a 16-foot by 20-foot quilt, and each patch is 4 square feet. So first, I need to find out how many patches he needs. Then, the cost structure is different depending on how many patches are purchased. The first 10 cost $10 each, and then any additional patches cost half that price. So maybe I should calculate the total area of the quilt first, then divide by 4 to get the number of patches. Let me do that.

The quilt is 16 feet by 20 feet, so the area is 16 * 20. Let's compute that: 16*20 is 320 square feet. Each patch is 4 square feet, so the total number of patches needed is 320 divided by 4. 320 ÷ 4 is 80. So James needs 80 patches.

Now, the cost. The first 10 patches cost $10

# Evaluation

## Resource usage

In [26]:
# Memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)

print(f'GPU = {gpu_stats.name}. Max memory = {max_memory} GB.')
print(f'{start_gpu_memory} GB of memory reserved.')

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.557 GB.
38.09 GB of memory reserved.


In [27]:
# Extract runtime info
last_log = trainer.state.log_history[-1] # Final memory and time stats
train_seconds = last_log['train_runtime']
samples_per_second = last_log.get('train_samples_per_second', None)

# Recompute GPU memory stats
used_memory   = round(torch.cuda.max_memory_reserved() / 1024**3, 2)
used_for_lora = round(used_memory - start_gpu_memory, 2)
used_pct      = round(used_memory / max_memory * 100, 2)
lora_pct      = round(used_for_lora / max_memory * 100, 2)

print(f'Training time: {train_seconds:.1f} seconds ({train_seconds / 60:.2f} minutes)')
if samples_per_second: print(f'Throughput: {samples_per_second:.1f} samples/second')
print(f'Peak VRAM usage: {used_memory} GB ({used_pct}% of max memory)')
print(f'VRAM for training: {used_for_lora} GB ({lora_pct}% of max memory)')

Training time: 1823.5 seconds (30.39 minutes)
Throughput: 0.9 samples/second
Peak VRAM usage: 38.09 GB (96.29% of max memory)
VRAM for training: 0.0 GB (0.0% of max memory)


## Verify LoRA is actually trained

In [28]:
example_text = 'What is the sqrt of 101?'
# example_text = 'Solve (x + 2)^2 = 0'
# example_text = "How many r's are in strawberry?"

sampling_params = SamplingParams(
    temperature=1.0,
    top_k=50,
    max_tokens=max_completion_length,
)
print(model.fast_generate( # Try the model without any GRPO trained
    example_text, sampling_params=sampling_params,
    lora_request=None
)[0].outputs[0].text)

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

 - Answers
Math and Arithmetic
What is the sqrt of 101?
Wiki User
∙ 2013-03-31 16:17:53
See answer (1)
Best Answer
Copy
sqrt of 101 is 10.05003
Wiki User
∙ 2013-03-31 16:17:53
This answer is:
Helpful ( 0)
Not Helpful ( 0)
Add a Comment
Study guides
Algebra
20 cards
A polynomial of degree zero is a constant term
The grouping method of factoring can still be used when only some of the terms share a common factor A True B False
The sum or difference of p and q is the of the x-term in the trinomial
A number a power of a variable or a product of the two is a monomial while a polynomial is the of monomials
➡️
See all cards
3.72
☆★☆★☆★☆★☆★
932 Reviews
Add your answer:
Earn + 20 pts
Q: What is the sqrt of 101?
Write your answer...
Submit
People also asked


In [29]:
tensors = {}
with safe_open(f'./{model_name}_grpo/adapter_model.safetensors', framework='pt') as f:
    for key in f.keys(): # Verify both A and B are non zero
        tensor = f.get_tensor(key)
        n_zeros = (tensor == 0).sum() / tensor.numel()
        assert(n_zeros.item() != tensor.numel())

In [ ]:
# Load the LoRA and test without using system prompt
# which should not (or minimal) affect the model's original reasoning ability
text = tokenizer.apply_chat_template(
    [{'role': 'user', 'content': example_text}],
    add_generation_prompt=True, tokenize=False,
)
print(model.fast_generate(
    text, sampling_params=sampling_params,
    lora_request=model.load_lora(f'./{model_name}_grpo'),
)[0].outputs[0].text)

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Okay, let me see. I need to find the square root of 101. Hmm, square root... So, I'm looking for a number that when multiplied by itself gives 101. Let me think. Let me start with some numbers I know. Like, 10 squared is 100, right? That's close to 101. So, maybe 10 is a bit too low. Let me try a little higher. Let's take 11. 11 squared is 121. Oh, that's too high. So, between 10 and 11, the square root has to be some number between those two. Let me try 10.5. Let's calculate 10.5 squared. 10.5 times 10.5. Let me do that step by step. 10*10 is 100, then 10*0.5 is 5, add those to get 105. Then 0.5*10 is 5, plus 5 more is 110. So, 10.5 squared is 110.25. That's a little high, but way above 101. Hmm, 10.4 maybe? Let me check. 10.4 squared. 10*10 is 100, 10*0.4 is 4, so 104. Plus 0.4*0.4 is 0.16, total 104.16. That's closer but still a bit high. Let me try 10.2. 10.2 squared. 10*10=100, 10*0.2=2, 0.2*10=2, plus 0.2*0.2=0.04, total 102.04. That's lower than 101. So, between 10.2 and 10.4. L

In [ ]:
# Test using system prompt
text = tokenizer.apply_chat_template([
    {'role': 'system', 'content': SYSTEM_PROMPT},
    {'role': 'user'  , 'content': example_text},
], add_generation_prompt=True, tokenize=False)

# Compare results with system prompt but without LoRA
print(model.fast_generate(
    text, sampling_params=sampling_params,
    lora_request=None,
)[0].outputs[0].text)

# Reasoning model is much better - it's not always correct, since we only trained it for an hour
# It'll be better if we extend the sequence length and train for longer
print(model.fast_generate(
    text, sampling_params=sampling_params,
    lora_request=model.load_lora(f'./{model_name}_grpo'),
)[0].outputs[0].text)

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

 We need to find the square root of 101.
We can estimate this using known perfect squares.
The perfect squares closest to 101 are $36^2 = 1296$ and $26^2 = 676$.
Since 101 is closer to 676, we estimate the square root of 101 to be around 10.<SOLUTION> The square root of 101 is around 10, as it lies between 10 and 11.<SOLUTION>


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Okay, so I need to find the square root of 101. Hmm, square roots... I remember that some numbers like 100 have a square root that's a whole number, which is 10, but 101 doesn't seem to be a perfect square. Let me think.

Wait, maybe I can estimate it. Since 10² is 100, the square root of 101 should be slightly more than 10. So around 10.1? Let me check 10.1 squared. 10.1² would be 100 plus 2*10*0.1 + 0.1², which is 100 + 2 + 0.01 = 102.01. That's a bit higher than 101. So 10.1 is too high. Let's try 10.05. 10.05² would be 100 + 2*10*0.05 + 0.05² = 100 + 1 + 0.0025 = 101.0025. That's very close to 101. So the square root must be 10.05. Or maybe even 10.0505? I don't think the decimal goes up that far. But let me check if there's a simpler way.

Is there a better method than trial and error? Maybe using a calculator? But since I don't have one right now, I'll stick with estimating. Alternatively, can 101 be expressed as a product of perfect squares? 101 divided by 100 is 1.01, which isn

## Performance on Test set

In [30]:
# test_dataset = load_dataset('openai/gsm8k', 'main', split=['test[:10%]'])
test_dataset = load_dataset('openai/gsm8k', 'main', split='test').map(process_dataset_sample)
test_texts = [
    tokenizer.apply_chat_template([
        {'role': 'system', 'content': SYSTEM_PROMPT},
        {'role': 'user', 'content': sample['prompt'][1]['content']},
    ], add_generation_prompt=True, tokenize=False)
for sample in test_dataset]
print(f'Testing samples:', len(test_dataset))

Map:   0%|          | 0/1319 [00:00<?, ? examples/s]

Testing samples: 1319


In [31]:
outputs_with_lora = model.fast_generate(
    test_texts, sampling_params=sampling_params,
    lora_request=model.load_lora(f'./{model_name}_grpo'),
)
outputs_without_lora = model.fast_generate(
    test_texts, sampling_params=sampling_params,
    lora_request=None,
)

Adding requests:   0%|          | 0/1319 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1319 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s…

Adding requests:   0%|          | 0/1319 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1319 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s…

In [32]:
# Compare the correct amount of using and not using LoRA
no_lora_correct_format_cnt = lora_correct_format_cnt = 0
no_lora_correct_answer_cnt = lora_correct_answer_cnt = 0
no_lora_correct_all_cnt = lora_correct_all_cnt = 0
num_test_samples = len(test_dataset)

for output_with_lora, output_without_lora, answer in zip(outputs_with_lora, outputs_without_lora, test_dataset['answer']):
    correct_format = match_format.search(output_with_lora.outputs[0].text)
    correct_answer = (guess := match_number.search(output_with_lora.outputs[0].text)) and guess.group(1) == answer
    correct_all = correct_format and correct_answer
    if correct_format: lora_correct_format_cnt += 1
    if correct_answer: lora_correct_answer_cnt += 1
    if correct_all: lora_correct_all_cnt += 1

    correct_format = match_format.search(output_without_lora.outputs[0].text)
    correct_answer = (guess := match_number.search(output_without_lora.outputs[0].text)) and guess.group(1) == answer
    correct_all = correct_format and correct_answer
    if correct_format: no_lora_correct_format_cnt += 1
    if correct_answer: no_lora_correct_answer_cnt += 1
    if correct_all: no_lora_correct_all_cnt += 1

pd.DataFrame({
    'Without LoRA': {
        'Correct Format': f'{no_lora_correct_format_cnt}/{num_test_samples} ({no_lora_correct_format_cnt / num_test_samples * 100:.2f}%)',
        'Correct Answer': f'{no_lora_correct_answer_cnt}/{num_test_samples} ({no_lora_correct_answer_cnt / num_test_samples * 100:.2f}%)',
        'Correct Both': f'{no_lora_correct_all_cnt}/{num_test_samples} ({no_lora_correct_all_cnt / num_test_samples * 100:.2f}%)',
    },
    'With LoRA': {
        'Correct Format': f'{lora_correct_format_cnt}/{num_test_samples} ({lora_correct_format_cnt / num_test_samples * 100:.2f}%)',
        'Correct Answer': f'{lora_correct_answer_cnt}/{num_test_samples} ({lora_correct_answer_cnt / num_test_samples * 100:.2f}%)',
        'Correct Both': f'{lora_correct_all_cnt}/{num_test_samples} ({lora_correct_all_cnt / num_test_samples * 100:.2f}%)',
    },
    'Improvement': {
        'Correct Format': f'+{lora_correct_format_cnt - no_lora_correct_format_cnt} ({(lora_correct_format_cnt - no_lora_correct_format_cnt) / num_test_samples * 100:.2f}%)',
        'Correct Answer': f'+{lora_correct_answer_cnt - no_lora_correct_answer_cnt} ({(lora_correct_answer_cnt - no_lora_correct_answer_cnt) / num_test_samples * 100:.2f}%)',
        'Correct Both': f'+{lora_correct_all_cnt - no_lora_correct_all_cnt} ({(lora_correct_all_cnt - no_lora_correct_all_cnt) / num_test_samples * 100:.2f}%)',
    }
}).T

,Correct Format,Correct Answer,Correct Both
Without LoRA,464/1319 (35.18%),309/1319 (23.43%),171/1319 (12.96%)
With LoRA,1263/1319 (95.75%),1016/1319 (77.03%),1015/1319 (76.95%)
Improvement,+799 (60.58%),+707 (53.60%),+844 (63.99%)


# Inference

In [ ]:
def generate_with_reasoning(questions, max_completion_length=512):
    conversations = [[                        # Format input using conversation template
        {'role': 'system', 'content': SYSTEM_PROMPT},
        {'role': 'user', 'content': question},
    ] for question in questions]

    prompts = [tokenizer.apply_chat_template( # Apply chat template and tokenize
        conversation,
        add_generation_prompt=True,         # Add assistant prompt
        tokenize=False,                     # Return string, not tokens
    ) for conversation in conversations]

    # Generate response with reasoning-optimized parameters
    inputs = tokenizer(prompts, return_tensors='pt', padding=True).to(model.device)
    start_time = time.time()
    with torch.no_grad():
        output_ids = model.generate(           # Generate response with reasoning-optimized parameters
            **inputs,
            max_new_tokens=max_completion_length,
            temperature=0.7,                # Balance creativity and consistency
            top_p=0.9,                      # Nucleus sampling for quality
            do_sample=True,                 # Enable sampling for varied reasoning paths
            pad_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.1,         # Reduce repetitive reasoning steps
            length_penalty=1.0,             # Neutral preference for response length
            early_stopping=True,            # Stop at natural completion
            streamer=TextStreamer(tokenizer, skip_prompt=True),
        )
    end_time = time.time()
    inference_duration = end_time - start_time
    num_generated_tokens = output_ids.shape[1] - inputs['input_ids'].shape[1]

    output_ids = output_ids[:, inputs['input_ids'][0].shape[-1]:output_ids.shape[-1]]
    responses = tokenizer.batch_decode(output_ids, skip_special_tokens=True) # Decode and extract only the generated portion
    return responses, inference_duration, num_generated_tokens

In [ ]:
test_dataset = load_dataset('openai/gsm8k', 'main', split='test').map(process_dataset_sample)
gsm8k_question = test_dataset[0]['question']
expected_answer = test_dataset[0]['answer']

print('Question:', gsm8k_question, '\nResponse:')
gsm8k_responses, inference_duration, num_generated_tokens = generate_with_reasoning([gsm8k_question], max_completion_length)
gsm8k_response = gsm8k_responses[0]
print('Inference time (secs):', inference_duration)
print('Generated tokens:', num_generated_tokens)

Question: Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market? 
Response:
Okay, let's try to figure out how much Janet makes from selling eggs each day at the farmer's market. Hmm, I need to start by figuring out how many eggs the ducks lay in total.

First, the ducks lay 16 eggs per day. But Janet uses some of those eggs herself. Specifically, she eats three eggs for breakfast every morning. So maybe I should subtract those three from the total? Let me think: if she lays 16 and eats 3, that would be 16 - 3 = 13 eggs left after breakfast. 

Then, she bakes muffins for her friends using four more eggs. Wait, so adding those back into the remaining count because they're used for baking? Yeah, like 13 (left after eating) + 4 (used for baking) equals 17. Then, subtr

In [ ]:
# Validate format compliance
has_solution = SOLUTION_START in gsm8k_response and SOLUTION_END in gsm8k_response
print('Reasoning section:', REASONING_END in gsm8k_response)
print('Solution section:', has_solution)

if has_solution: # Check answer accuracy if solution section exists
    try:
        solution_text = gsm8k_response.split(SOLUTION_START)[1].split(SOLUTION_END)[0].strip()
        extracted_number = ''.join(filter(str.isdigit, solution_text))
        expected_number = ''.join(filter(str.isdigit, expected_answer))
        print('Extracted:', solution_text)
        print('Expected:', expected_answer)
        print('Correct:', extracted_number == expected_number)
    except:
        print('Could not extract solution')

Reasoning section: True
Solution section: True
Extracted: 18
Expected: 18
Correct: True
